In [8]:
import os
from pathlib import Path
import PyPDF2
import shutil
import csv
from datetime import datetime
import itertools
import pandas as pd

In [2]:
base_dir = Path(r"Z:\uat\ach_eobr\DailyArchive")
password = "Unlock321"
report_file = base_dir / "protected_files_report.csv"

def find_latest_folder(base_path):
    """Find the latest date-stamped folder in the base directory."""
    all_folders = [f for f in base_path.iterdir() if f.is_dir()]
    latest_folder = max(all_folders, key=os.path.getmtime)
    return latest_folder

def password_protect_pdf(input_pdf, output_pdf, password):
    """Password protect the input PDF and save it to the output path."""
    try:
        with open(input_pdf, "rb") as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            writer = PyPDF2.PdfWriter()

            # Copy pages and apply password
            for page_num in range(len(reader.pages)):
                writer.add_page(reader.pages[page_num])

            writer.encrypt(password)

            # Save the protected PDF
            with open(output_pdf, "wb") as output_file:
                writer.write(output_file)
        
        return True
    except Exception as e:
        print(f"Failed to password protect {input_pdf}: {e}")
        return False

def count_pdf_pages(pdf_path):
    """Return the number of pages in the PDF."""
    try:
        with open(pdf_path, "rb") as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            return len(reader.pages)
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return 0

def generate_report(report_path, report_data):
    """Generate a CSV report for the protected files."""
    with open(report_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Vendor", "File Path", "Status", "Timestamp"])
        for row in report_data:
            writer.writerow(row)

def main():
    # Find the latest date folder
    latest_folder = find_latest_folder(base_dir)

    # Report data collection
    report_data = []

    # Loop through each vendor folder
    for vendor_folder in latest_folder.iterdir():
        if vendor_folder.is_dir():
            # Find the _PAID.pdf and _ZERO.pdf files
            pdf_files = itertools.chain(
                vendor_folder.glob("*_PAID.pdf"), 
                vendor_folder.glob("*_ZERO_FILE.pdf")
            )

            for pdf_file in pdf_files:
                try:
                    # Count the pages in the PDF
                    num_pages = count_pdf_pages(pdf_file)

                    # Only protect PDFs with more than one page
                    if num_pages > 1:
                        # Define the output path in the same vendor folder
                        destination_pdf = pdf_file.with_name(f"{pdf_file.stem}_PROTECTED.pdf")

                        # Protect the PDF
                        success = password_protect_pdf(pdf_file, destination_pdf, password)

                        # Add to report
                        status = "Success" if success else "Failed"
                        report_data.append([vendor_folder.name, str(destination_pdf), status, datetime.now().isoformat()])
                    else:
                        # Log that the file was skipped due to being a single-page PDF
                        report_data.append([vendor_folder.name, str(pdf_file), "Skipped (single-page)", datetime.now().isoformat()])

                except Exception as e:
                    print(f"Error processing {pdf_file}: {e}")
                    report_data.append([vendor_folder.name, str(pdf_file), "Error", datetime.now().isoformat()])

    # Generate the report
    generate_report(report_file, report_data)
    print(f"Report generated: {report_file}")

if __name__ == "__main__":
    main()

Error reading Z:\work\ach_eobr\DailyArchive\20241119\JEWETT\JEWETT_SURGERY_PAID.pdf: Cannot read an empty file
Error reading Z:\work\ach_eobr\DailyArchive\20241119\JEWETT\JEWETT_SURGERY_ZERO_FILE.PDF: Cannot read an empty file
Report generated: Z:\work\ach_eobr\DailyArchive\protected_files_report.csv
